# Grouped Computations

In this lesson, we discuss how to do scientific computations with defined "groups" of data
within our xarray objects. Our learning goals are as follows:

- Perform "split / apply / combine" workflows in Xarray using `groupby`,
  including
  - reductions within groups
  - transformations on groups
- Use `resample` to change the time frequency of the data


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# don't expand data by default
xr.set_options(display_expand_data=False, display_expand_attrs=False)

%config InlineBackend.figure_format='retina'

## Example Dataset

First we load a dataset. We will use the
[NOAA Extended Reconstructed Sea Surface Temperature (ERSST) v5](https://www.ncei.noaa.gov/products/extended-reconstructed-sst)
product, a widely used and trusted gridded compilation of of historical data
going back to 1854.

In [ ]:
ds = xr.tutorial.load_dataset("ersstv5")
ds

## Groupby

Xarray copies Pandas' very useful groupby functionality, enabling the "split /
apply / combine" workflow on xarray DataArrays and Datasets.

Let's examine a timeseries of SST at
a single point.


In [ ]:
ds.sst.sel(lon=300, lat=50).plot();

As we can see from the plot, the timeseries at any one point is totally
dominated by the seasonal cycle. We would like to remove this seasonal cycle
(called the "climatology") in order to better see the long-term variaitions in
temperature. We can accomplish this using **groupby**.

Before moving forward, we note that xarray correctly parsed the time index,
resulting in a Pandas datetime index on the time dimension.


In [ ]:
ds.time

The syntax of Xarray's groupby is almost identical to Pandas.


In [ ]:
?ds.groupby

### Identifying groups

The most important argument is `group`: this defines the unique values or labels we will
us to "split" the data for grouped analysis. We can pass either a DataArray or a
name of a variable in the dataset. Let's first use a DataArray. 

Just like with
Pandas, we can use the time index to extract specific components of dates and
times. Xarray uses a special syntax for this `.dt`, called the
[DatetimeAccessor](https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html?highlight=DatetimeAccessor). See the [documentation](https://docs.xarray.dev/en/stable/user-guide/time-series.html#datetime-components) for more


In [ ]:
ds.time.dt

In [ ]:
ds.time.dt.month

In [ ]:
ds.time.dt.year

### Split step

We can use these arrays in a groupby operation:


In [ ]:
gb = ds.groupby(ds.time.dt.month)
gb

Xarray also offers a more concise syntax when the variable you're grouping on is
already present in the dataset. This is identical to the previous line:


In [ ]:
gb = ds.groupby("time.month")
gb

`gb` is a DatasetGroupBy object. It represents a GroupBy operation and helpfully tells us the unique "groups" or labels found during the split step.


```{tip}

Xarrays' computation methods (`groupby`, `groupby_bins`, `rolling`, `coarsen`, `weighted`) all return special objects that represent the basic underlying computation pattern. For e.g. `gb` above is a `DatasetGroupBy` object that represents monthly groupings of the data in `ds` . It is usually helpful to save and reuse these objects for multiple operations (e.g. a mean and standard deviation calculation).
```

### Apply & Combine

Now that we have groups defined, it's time to "apply" a calculation to the
group. Like in Pandas, these calculations can either be:

- _aggregation_ or _reduction_: reduces the size of the group
- _transformation_: preserves the group's full size

At then end of the apply step, xarray will automatically combine the aggregated
/ transformed groups back into a single object.

#### Aggregations or Reductions

Most commonly, we want to perform a reduction operation like `sum` or `mean` on our groups. Xarray conveniently provides these reduction methods on Groupby objects for both [DataArrays and Datasets](https://docs.xarray.dev/en/stable/api.html#groupby-objects).

Here we calculate the monthly mean.

In [ ]:
ds_mm = gb.mean()
ds_mm

So we did what we wanted to do: calculate the climatology at every point in the
dataset. Let's look at the data a bit.

_Climatology at a specific point in the North Atlantic_


In [ ]:
ds_mm.sst.sel(lon=300, lat=50).plot();

_Zonal Mean Climatology_


In [ ]:
ds_mm.sst.mean(dim="lon").plot.contourf(x="month", levels=12, vmin=-2, vmax=30);

_Difference between January and July Climatology_


In [ ]:
(ds_mm.sst.sel(month=1) - ds_mm.sst.sel(month=7)).plot(vmax=10);

#### Custom Aggregations

The most fundamental way to apply a function and combine the results together to use the `.map` method.

In [ ]:
?gb.map

`.map` accepts as its argument a function that expects and returns xarray
objects. We define a custom function. This function takes a single argument--the
group dataset--and returns a new dataset to be combined:

In [ ]:
def time_mean(a):
    return a.mean(dim="time")


gb.map(time_mean)

This is identical to `gb.mean()`

#### Apply by iteration

We can manually iterate over the group. The
iterator returns the key (group name) and the value (the actual dataset
corresponding to that group) for each group.


You could apply any function you want in the loop but you would have to manually [combine](https://docs.xarray.dev/en/stable/user-guide/combining.html) the results together.

In [ ]:
for group_name, group_ds in gb:
    # stop iterating after the first loop
    break
print(group_name)
group_ds

#### Transformations

Now we want to _remove_ this climatology from the dataset, to examine the
residual, called the _anomaly_, which is the interesting part from a climate
perspective. Removing the seasonal climatology is a perfect example of a
transformation: it operates over a group, but doesn't change the size of the
dataset. Here is one way to code it


In [ ]:
def remove_time_mean(x):
    return x - x.mean(dim="time")


ds_anom = ds.groupby("time.month").map(remove_time_mean)
ds_anom

Xarray makes these sorts of transformations easy by supporting _groupby
arithmetic_. This concept is easiest explained with an example:


In [ ]:
gb = ds.groupby("time.month")
ds_anom = gb - gb.mean()
ds_anom

Now we can view the climate signal without the overwhelming influence of the
seasonal cycle.

_Timeseries at a single point in the North Atlantic_


In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot();

_Difference between Jan. 1 2018 and Jan. 1 1970_


In [ ]:
(ds_anom.sel(time="2018-01-01") - ds_anom.sel(time="1970-01-01")).sst.plot();

```{exercise} 
:label: annual-mean

Using `groupby`, plot the annual mean time series of SST at 300°E, 50°N
```
````{solution} annual-mean
:class: dropdown
```python
ds.groupby("time.year").mean().sst.sel(lon=300, lat=50).plot();
```
````

## Resample

Resampling means changing the time frequency of data, usually reducing to a coarser frequency: e.g. converting daily frequency data to monthly frequency data using `mean` to reduce the values. This operation can be thought of as a groupby operation where each group is a single month of data. Resampling can be applied only to time-index dimensions. 

First note that `ds_anom` has data at monthly frequency (i.e. one point every month).

In [ ]:
ds_anom.time

Here we compute the five-year mean along the `time` dimension by passing `time='5Y'`. `'5Y'` is a special frequency string. Xarray uses pandas to convert such a frequency string to a groupby operation. See the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) for how to specify a different frequency.

In [ ]:
resample_obj = ds_anom.resample(time="5Y")
resample_obj

```{note}
`resample` only works with proper datetime64 coordinate labels. Note the `dtype` of `time` in the repr above.
```

Resampling objects are exactly like groupby objects and allow reductions, iteration, etc.

In [ ]:
ds_anom_resample = resample_obj.mean()
ds_anom_resample

In [ ]:
for label, group in resample_obj:
    break
print(label, "\n\n", group)

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot()
ds_anom_resample.sst.sel(lon=300, lat=50).plot(marker="o");

```{exercise}
:label: resample-mean

Using `resample`, plot the annual mean time series of SST at 300°E, 50°N.

Compare this output to the groupby output. What differences do you see?
```
````{solution} resample-mean
:class: dropdown
```python
resampled = ds.resample(time='Y').mean().sst.sel(lon=300, lat=50)
resampled.plot();
```
````

## GroupBy vs Resample 

Let's compare the grouped and resampled outputs.


1. Note the different dimension names: when grouped, `time` is renamed to `year`. When resampled, the `time` dimension name is preserved
2. The values for `year` are integers, while those for `resampled.time` are timestamps, similar to the input dataset
3. But all values are equal

In [ ]:
from IPython.display import display_html

grouped = ds.groupby("time.year").mean().sst.sel(lon=300, lat=50)
resampled = ds.resample(time='Y').mean().sst.sel(lon=300, lat=50)
display_html(grouped)
display_html(resampled)

In [ ]:
np.array_equal(grouped.data, resampled.data)

## Going further

1. See the documentation on [groupby](https://docs.xarray.dev/en/stable/user-guide/groupby.html) and [resample](https://docs.xarray.dev/en/stable/user-guide/time-series.html#resampling-and-grouped-operations)
2. Follow the tutorial on [high-level computation patterns](https://tutorial.xarray.dev/intermediate/01-high-level-computation-patterns.html)